# script to capture images in a loop. 

## make sure, in the web interface, you close the camera. Run the cells one by one.

In [1]:
#some imports
import time
from PIL import Image
import numpy as np
import pandas as pd
import pathlib
from picamera2 import Picamera2, Preview
import pprint
from ipywidgets import interact, interactive
from IPython.display import display
import matplotlib.pyplot as plt
from characterization_ams.stats_engine import stats
from characterization_ams.emva import emva
from characterization_ams.standard_tests import ptc
# from char_crunch.processing.process import Process


In [2]:
amount = 10 #numbers of pictures to capture per setting
bit_mode = 12
analog_gain = 1
exposure = 500 #in us #np.arange(1000, 10000, 500, dtype=int) #start, stop, step - this must be an array type. can also be, [100,200,300,400] etc..
min_exposure = 100
max_power = 1
min_exp = 1000
max_exp = 20000
exposures = np.linspace(min_exp, max_exp, 20)
board_id = 'csp_revB_385'


#select mode a few cells below.
TIFF = True
UPLOAD_FOLDER = pathlib.Path("./images"+board_id)
UPLOAD_FOLDER.mkdir(parents=False, exist_ok=True)

temp_save_dir = UPLOAD_FOLDER
final_save_dir = UPLOAD_FOLDER

In [3]:
#view camera model
pprint.pprint(Picamera2.global_camera_info() ) #before init ;
#print all sensor modes
with Picamera2() as picam2:
    modes = picam2.sensor_modes
    pprint.pprint(picam2.sensor_modes)

[{'Id': '/base/soc/i2c0mux/i2c@1/mira016@36',
  'Location': 2,
  'Model': 'mira016',
  'Rotation': 0}]
[{'bit_depth': 10,
  'crop_limits': (0, 0, 400, 400),
  'exposure_limits': (41, 1000396, None),
  'format': SGRBG10_CSI2P,
  'fps': 357.14,
  'size': (400, 400),
  'unpacked': 'SGRBG10'},
 {'bit_depth': 12,
  'crop_limits': (0, 0, 400, 400),
  'exposure_limits': (41, 1000396, None),
  'format': SGRBG12_CSI2P,
  'fps': 199.6,
  'size': (400, 400),
  'unpacked': 'SGRBG12'},
 {'bit_depth': 8,
  'crop_limits': (0, 0, 400, 400),
  'exposure_limits': (41, 1000396, None),
  'format': SGRBG8,
  'fps': 357.14,
  'size': (400, 400),
  'unpacked': 'SGRBG8'}]


[100:31:21.726769173] [51529]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2024-05-28T14:42:57+02:00)
[100:31:21.730756965] [51551]  WARN CameraSensorProperties camera_sensor_properties.cpp:573 No static properties available for 'mira016'
[100:31:21.730800779] [51551]  WARN CameraSensorProperties camera_sensor_properties.cpp:575 Please consider updating the camera sensor properties database
[100:31:21.741708184] [51551]  WARN RPI raspberrypi.cpp:1357 Mismatch between Unicam and CamHelper for embedded data usage!
[100:31:21.742748580] [51551]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/mira016@36 to Unicam device /dev/media4 and ISP device /dev/media0
[100:31:21.749394696] [51529]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2024-05-28T14:42:57+02:00)
[100:31:21.753152287] [51554]  WARN CameraSensorProperties camera_sensor_properties.cpp:573 No static properties available for 'mira016'
[100:31:21.75319693

# select mode here in the dropdown

In [4]:
for mode in modes:
    if mode['bit_depth']==bit_mode:
        break
selected_mode=mode
all_images = []

with Picamera2() as picam2:
    
    preview_config = picam2.create_preview_configuration(main={"size": selected_mode["size"]},
    raw={"format": selected_mode["unpacked"],
        "size": selected_mode["size"],
    })
    
    picam2.configure(preview_config)
    
    picam2.start()
    
    im_data = pd.DataFrame()
    # im_data = pd.read_csv(Path(temp_save_dir / r'image_data.csv'))
    
    
    
    exposure = min_exp
    
    # get dark image
    picam2.set_controls({"ExposureTime": exposure , "AnalogueGain": analog_gain})
    
    a = input('turn off light')
    
    #### CAPTURE
    size = selected_mode["size"]
    
    width = size[0]
    height = size[1]
    im_stack = []
    for i in range(amount):
        if bit_mode == 8:
            image = picam2.capture_array("raw").view(np.uint8)
        else:
            image = picam2.capture_array("raw").view(np.uint16)
        im_stack.append(image[0:height, 0:width])
        # if TIFF:
        #     # save images    
        #     pilim = Image.fromarray(image)
        #     filename = str(f"{UPLOAD_FOLDER}/img_exposure{exposure}_gain{analog_gain}{i}.tiff")
        #     pilim.save(filename)
        
    # get & save images
    name = f'bm={bit_mode}_ag={analog_gain}_fc={amount}_exp={round(exposure, 1)}_dark.npz'
    temp_im_dir = UPLOAD_FOLDER
    temp_im_path = pathlib.Path(temp_im_dir / name)
    np.savez(str(temp_im_path), im_stack)
    
    all_images.append(np.stack(im_stack))
    
    
    # add to image data
    temp = {'Analog Gain': [analog_gain],
            'Bit Mode': [bit_mode],
            'Exposure [ms]': [min_exp],
            'Frame Count': [amount],
            'Power [uW/cm^2]': [0],
            'Exposure [uW/cm^2*s]': [0],
            'Image Path': [str(pathlib.Path(UPLOAD_FOLDER / name))]}
    temp = pd.DataFrame.from_dict(temp)
    
    # # concat raw to im_data
    im_data = pd.concat([im_data, temp], ignore_index=True).reset_index(drop=True)
    im_data_path = pathlib.Path(UPLOAD_FOLDER / r'image_data.csv')
    im_data.to_csv(im_data_path, index=False)
    print('done')

    
    a = input('turn on light')
    # sweep exposures
    for exposure in exposures:
        picam2.set_controls({"ExposureTime": int(exposure) , "AnalogueGain": analog_gain})
        
        # cam.states.cycle_states()
        print(f'exposure is {exposure}')
    
        
        time.sleep(0.1)
    
        # set light source
        power = max_power / analog_gain
    #     source.set_channel(channel_identifier=500, power=power, set_wl=True)
    
        # get & save images
    
        im_stack = []
        for i in range(amount):
            if bit_mode == 8:
                image = picam2.capture_array("raw").view(np.uint8)
            else:
                image = picam2.capture_array("raw").view(np.uint16)
            im_stack.append(image[0:height, 0:width])
            # if TIFF:
            #     # save images    
            #     pilim = Image.fromarray(image)
            #     filename = str(f"{UPLOAD_FOLDER}/img_exposure{exposure}_gain{analog_gain}{i}.tiff")
            #     pilim.save(filename)
    
        
        name = f'bm={bit_mode}_ag={analog_gain}_fc={amount}_exp={round(exposure, 1)}_illum.npz'
        temp_im_dir = UPLOAD_FOLDER
        temp_im_path = pathlib.Path(temp_im_dir / name)
        np.savez(str(temp_im_path), im_stack)
        all_images.append(np.stack(im_stack))

    
        # add to image data
        temp = {'Analog Gain': [analog_gain],
                'Bit Mode': [bit_mode],
                'Exposure [ms]': [exposure],
                'Frame Count': [amount],
                'Power [uW/cm^2]': ['source.power'],
                'Exposure [uW/cm^2*s]': [exposure / 1000 * power],
                'Image Path': [str(pathlib.Path(final_save_dir / name))]}
        temp = pd.DataFrame.from_dict(temp)
    
        # concat raw to im_data
        im_data = pd.concat([im_data, temp], ignore_index=True).reset_index(drop=True)
        im_data_path = pathlib.Path(temp_save_dir / r'image_data.csv')
        im_data.to_csv(im_data_path, index=False)

[100:31:22.052515799] [51529]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70-dirty (2024-05-28T14:42:57+02:00)
[100:31:22.057196861] [51558]  WARN CameraSensorProperties camera_sensor_properties.cpp:573 No static properties available for 'mira016'
[100:31:22.057245176] [51558]  WARN CameraSensorProperties camera_sensor_properties.cpp:575 Please consider updating the camera sensor properties database
[100:31:22.070279299] [51558]  WARN RPI raspberrypi.cpp:1357 Mismatch between Unicam and CamHelper for embedded data usage!
[100:31:22.072359444] [51558]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/mira016@36 to Unicam device /dev/media4 and ISP device /dev/media0
[100:31:22.079026338] [51529]  INFO Camera camera.cpp:1028 configuring streams: (0) 400x400-XBGR8888 (1) 400x400-SGRBG12
[100:31:22.079682442] [51558]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/mira016@36 - Selected sensor format: 400x400-SGRBG12_1X12 - Selected uni

turn off light 


done


turn on light 


exposure is 1000.0
exposure is 2000.0
exposure is 3000.0
exposure is 4000.0
exposure is 5000.0
exposure is 6000.0
exposure is 7000.0
exposure is 8000.0
exposure is 9000.0
exposure is 10000.0
exposure is 11000.0
exposure is 12000.0
exposure is 13000.0
exposure is 14000.0
exposure is 15000.0
exposure is 16000.0
exposure is 17000.0
exposure is 18000.0
exposure is 19000.0
exposure is 20000.0


In [5]:
im_data_path


PosixPath('imagescsp_revB_385/image_data.csv')

In [8]:
outp= ptc.ptc(img_stack_list=all_images,
                           df=im_data,
                           exp_col='Exposure [ms]', 
                          pixel_area =1,
                             shading_dim = 4)

could not calculate 50% point! Setting to half dataset size
Could not make linearity_fit, Error expected non-empty vector for x


KeyError: 'powermeter.wavelength'

In [ ]:
all_images[0].shape
